# Importing Libraries


In [9]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
import os

# Loading Documents

In [10]:
loader = TextLoader('./data/test_text.txt') # to load text document
documents = loader.load()

# Creating Chunks

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=40)
chunks = text_splitter.split_documents(documents)

In [24]:
chunks

[Document(page_content='TEXT FILES ARE AWESOME AS THEY CONTAIN TEXTS AND STUPID TEXTS. THEY ALSO CONTAIN', metadata={'source': './data/test_text.txt'}),
 Document(page_content='AND STUPID TEXTS. THEY ALSO CONTAIN WHATEVER HAHAHA', metadata={'source': './data/test_text.txt'})]

# Loading Models and Embeddings

In [15]:
# Creating Embeddings
embeddings = LlamaCppEmbeddings(
    model_path="models/llama-2-7b-chat.gguf.q4_0.bin",
    verbose=True
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from models/llama-2-7b-chat.gguf.q4_0.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = llama-2-7b-chat.ggmlv3.q8_0.bin
llama_model_loader: - kv   2:                        general.description str              = converted from legacy GGJTv3 MOSTLY_Q...
llama_model_loader: - kv   3:                          general.file_type u32              = 7
llama_model_loader: - kv   4:                       llama.context_length u32              = 2048
llama_model_loader: - kv   5:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   6:                          llama.block_count u32              = 32
llama_model_loader

llama_model_loader: - kv  10:              llama.attention.head_count_kv u32              = 32
llama_model_loader: - kv  11:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llama_model_loader: - kv  12:                       tokenizer.ggml.model str              = llama
llama_model_loader: - kv  13:                      tokenizer.ggml.tokens arr[str,32000]   = ["<unk>", "<s>", "</s>", "<0x00>", "<...
llama_model_loader: - kv  14:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  15:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  16:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  17:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  18:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - typ

In [20]:
chunks

[Document(page_content='TEXT FILES ARE AWESOME AS THEY CONTAIN TEXTS AND', metadata={'source': './data/test_text.txt'}),
 Document(page_content='THEY CONTAIN TEXTS AND STUPID TEXTS. THEY ALSO', metadata={'source': './data/test_text.txt'}),
 Document(page_content='STUPID TEXTS. THEY ALSO CONTAIN WHATEVER HAHAHA', metadata={'source': './data/test_text.txt'})]

In [4]:
x = embeddings.embed_documents(['yo man'])


llama_print_timings:        load time =   21385.64 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   21364.71 ms /     3 tokens ( 7121.57 ms per token,     0.14 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   21386.62 ms /     4 tokens


In [5]:
x

[[0.2832351326942444,
  -0.06352390348911285,
  0.013480950146913528,
  0.0218911562114954,
  -0.22481229901313782,
  -0.052022822201251984,
  0.1526179015636444,
  -0.21852900087833405,
  0.06574110686779022,
  0.15532684326171875,
  0.16566382348537445,
  0.047549884766340256,
  0.08735382556915283,
  0.19773359596729279,
  -0.13580985367298126,
  -0.0338468998670578,
  -0.3194659948348999,
  0.030069515109062195,
  -0.01717807911336422,
  -0.1854475736618042,
  0.09718301892280579,
  0.10907355695962906,
  0.17792636156082153,
  0.05986124277114868,
  -0.06649996340274811,
  0.05377795919775963,
  -0.2874511182308197,
  0.2597196400165558,
  -0.19092048704624176,
  0.13096322119235992,
  -0.09838007390499115,
  0.12735475599765778,
  -0.1199692040681839,
  -0.18760336935520172,
  -0.20822688937187195,
  -0.023977382108569145,
  -0.19288162887096405,
  -0.13590095937252045,
  0.05979907512664795,
  -0.09605475515127182,
  -0.060271866619586945,
  -0.0808110162615776,
  -0.09020612388

In [10]:
len(x)

1

In [12]:
a = [[[1,2,3]]]

In [15]:
a[0][0]

[1, 2, 3]

In [18]:
[list(map(float, e)) for e in a[0]]


[[1.0, 2.0, 3.0]]

In [16]:
x[0][0]

[0.2832351326942444,
 -0.06352390348911285,
 0.013480950146913528,
 0.0218911562114954,
 -0.22481229901313782,
 -0.052022822201251984,
 0.1526179015636444,
 -0.21852900087833405,
 0.06574110686779022,
 0.15532684326171875,
 0.16566382348537445,
 0.047549884766340256,
 0.08735382556915283,
 0.19773359596729279,
 -0.13580985367298126,
 -0.0338468998670578,
 -0.3194659948348999,
 0.030069515109062195,
 -0.01717807911336422,
 -0.1854475736618042,
 0.09718301892280579,
 0.10907355695962906,
 0.17792636156082153,
 0.05986124277114868,
 -0.06649996340274811,
 0.05377795919775963,
 -0.2874511182308197,
 0.2597196400165558,
 -0.19092048704624176,
 0.13096322119235992,
 -0.09838007390499115,
 0.12735475599765778,
 -0.1199692040681839,
 -0.18760336935520172,
 -0.20822688937187195,
 -0.023977382108569145,
 -0.19288162887096405,
 -0.13590095937252045,
 0.05979907512664795,
 -0.09605475515127182,
 -0.060271866619586945,
 -0.0808110162615776,
 -0.09020612388849258,
 0.41464415192604065,
 0.1023268103

In [8]:

vector_store = Chroma.from_documents(chunks, embeddings)

# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# # Loading the Model
# llm = LlamaCpp(
#     model_path="models/llama-2-7b-chat.gguf.q4_0.bin",
#     temperature=0.75,
#     max_tokens=2000,
#     top_p=1,
#     callback_manager=callback_manager,
#     verbose=False, 
#     n_ctx=2048
# )



llama_print_timings:        load time =   21385.64 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =   31297.22 ms /    43 tokens (  727.84 ms per token,     1.37 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   31334.38 ms /    44 tokens


In [ ]:

retriever=vector_store.as_retriever()
chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

question = st.text_input('Input your question')
if question:
    response = chain.run(question)
    st.write(response)

    
##############################

import streamlit as st

def main():
    st.title("Text File Viewer")

    uploaded_file = st.file_uploader("Upload a text file", type=["txt"])
    
    if uploaded_file is not None:
        file_contents = uploaded_file.read().decode("utf-8")
        st.text_area("File Contents", value=file_contents, height=500)

if __name__ == "__main__":
    main()
